In [ ]:
import json
from py2neo import Graph, Node, Relationship 

#graph = Graph("http://matlaber5.media.mit.edu:1234", auth=('neo4j', 'password'))
graph = Graph("bolt://neo4j:7687")

n_nodes = graph.database.primitive_counts['NumberOfNodeIdsInUse']
#n_relationships = graph.database.primitive_counts['NumberOfRelationshipIdsInUse']
#print("Connected to graph database with {:,} nodes and {:,} relationships!".format
     #(n_nodes, n_relationships))

In [ ]:
graph.run("MATCH (b) RETURN b")

In [ ]:
# Run STREAMING PageRank (1) on each year from 1800 to 2020
import pandas as pd
import numpy as np
import time

start_time = time.time()
start_year, end_year, step = 1985, 2010, 5
dfs = []
for year in range(start_year, end_year+1, step):
    
    # < IS MUCH FASTER THAN <=
    print("Running PageRank on works from < {}...".format(year), end=" ")
    query_start_time = time.time()
    query = """
    CALL algo.pageRank.stream(
    'MATCH (p:Quanta) WHERE p.year < {} RETURN id(p) as id',
    'MATCH (p1:Quanta)-[:CITES]->(p2:Quanta) RETURN id(p1) as source, id(p2) as target',
    {{graph:'cypher', iterations:20, write:false, concurrency:20}})
    YIELD node, score
    WITH 
        *,
        node.id AS id, 
        node.title AS title, 
        node.lang AS lang, 
        node.year AS year, 
        node.keywords AS keywords, 
        node.fos AS fos, 
        node.publisher AS publisher,
        score AS page_rank
    ORDER BY page_rank DESC
    LIMIT 10000
    RETURN 
        id,
        title,
        lang, 
        year, 
        keywords, 
        fos, 
        publisher,
        page_rank;    
    """.format(year,year,year)
    
    #graph.run(query)

    print(query)
    df = graph.run(query).to_data_frame()
    df['year'] = year
    dfs.append(df)
    print("Done ({:.2f} minutes).".format((time.time()-query_start_time)/60))
    
print("Finished all calculations in {:.2f} minutes.".format((time.time()-start_time)/60))


In [ ]:
df

In [ ]:
all_results = pd.concat(dfs)
all_results_path = '/tmp/data/result/allResults_{}-{}-{}.csv'.format(start_year, end_year,step)
print("Writing all results to {}...".format(all_results_path), end=" ")
all_results.to_csv(path_or_buf=all_results_path, sep=",", header=True, index=True)
print("Done.")



In [ ]:
df['title']

In [ ]:
import numpy as np

# all_results['title_clean'] = all_results['title'].str.replace(',',' ')
all_results['primary_field'] = all_results['fos'].apply(lambda x: x[0] if np.all(pd.notnull(x)) else None)
result = all_results.pivot_table(index=['title','primary_field','publisher'], columns='year', values='impact')    

file_path = '/tmp/data/result/impact_{}-{}-{}.csv'.format(start_year, end_year,step)
print("Writing results to {}...".format(file_path), end=" ")
result.to_csv(path_or_buf=file_path, sep=",", header=True, index=True)
print("Done.")



In [ ]:
result

In [ ]:
# # Graph loading (notes; do not execute)
# # // Load graph
CALL algo.graph.load('my-graph','Label','REL_TYPE',{graph:'heavy',..other config...})
  YIELD name, graph, direction, undirected, sorted, nodes, loadMillis, alreadyLoaded,
        nodeWeight, relationshipWeight, nodeProperty, loadNodes, loadRelationships;

# # // Info on loaded graph
# CALL algo.graph.info('my-graph')
#   YIELD name, type, exists, removed, nodes;

# # // Use graph
# CALL algo.pageRank(null,null,{graph:'my-graph',...})


# # // Remove graph
# CALL algo.graph.remove('my-graph')
#   YIELD name, type, exists, removed, nodes;